In [38]:
import os 
import openai 
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.schema.runnable import RunnablePassthrough




In [3]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [11]:
messages = [
     SystemMessage(
        content="""You're an assistant knowledgeable about
         driving liscenes in dubai. Only answer driving liscene related questions."""
     ),
     HumanMessage(content="can u tell more information about liquar?"),
 ]

chat_model.invoke(messages)

AIMessage(content="I'm here to help with questions related to driving licenses in Dubai. If you have any queries about obtaining or renewing a driving license in Dubai, feel free to ask!", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 45, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

In [13]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't kno

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_template.format(context=context, question=question)


"Human: Your job is to use patient\nreviews to answer questions about their experience at a hospital\nUse the following context to answer questions. Be as detailed\nas possible, but don't make up any information that's not\nfrom the context. If you don't know an answer, say you don't kno\n\nI had a great stay!\n\nDid anyone have a positive experience?\n"

In [22]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """ Studends will query about driving rules in dubai.You have information 
on the theory test which is conducting by RTA DUBAI. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

context = "road rules in dubai"
question = "what are the rules when turn left in roundabout"
#question = "what are the plavces to buy alcohol in dubai"

review_prompt_template.format_messages(context=context, question=question)

review_chain = review_prompt_template | chat_model

In [23]:
review_chain.invoke({"context": context, "question": question})

AIMessage(content='In Dubai, when turning left in a roundabout, you should approach the roundabout in the left lane and indicate your intention to turn left. You must give way to any vehicles already in the roundabout, including those coming from your right. Once it is safe to do so, you can enter the roundabout and continue in a counter-clockwise direction until you reach your exit. Remember to always follow the road signs and markings within the roundabout to ensure a safe and smooth flow of traffic.', response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 98, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

In [32]:

loader = PyPDFLoader('./data/uae_driving.pdf')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [29]:
vector_db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [37]:
retriever = vector_db.as_retriever(search_kwargs={"k": 10})


In [39]:
review_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | chat_model
    | StrOutputParser()
)

In [44]:
question = """what are the plavces to buy alcohol in duba"""
review_chain.invoke(question)

"I'm sorry, but the information provided does not mention specific places to buy alcohol in Dubai. If you have any other questions related to driving rules or regulations in Dubai, feel free to ask!"

In [43]:
question = """what are the rules to follow in intersection?"""
review_chain.invoke(question)

'When approaching an intersection, there are several rules to follow:\n\n1. **Stop Line**: If there is a Stop sign or traffic signals at the intersection, you must come to a complete stop before the white Stop line painted across your lane. Only proceed if you have a green traffic signal and ensure the intersection is clear and safe.\n\n2. **Give Way Line**: If there is a Give Way sign, you must yield to any vehicle or pedestrian crossing your path. Slow down and stop if necessary to avoid a collision. Always ensure the intersection is clear and safe before proceeding.\n\n3. **Pedestrian Crossings**: When turning left or right at any intersection, you must give way to any pedestrians crossing the road you are turning into. If there are pedestrian crossing signals or police officers, follow their instructions. If not, slow down while approaching and give way to pedestrians.\n\n4. **Traffic Control Signs**: Obey Stop and Give Way signs at intersections. Do not force your way in, prioriti